# <span style="color:skyblue"><u>Práctica UD5 N.N. con dataset Mnist</u></span>

Salvador Lopez

# <span style="color:cornflowerblue"><u>1.-Crea un modelo de clasificación del dataset MNIST con PyTorch con redes neuronales sin capas convolucionales, intentando mejorar todo lo posible su exactitud.</u> </span>


## 1.1.- Importamos las librerias necesarias

Se importan las librerias de PyTorch necesarias para construir la Red Neuronal. 

Se descarga el dataset Mnist separando el grupo de entrenamiento y test y transformando las imagenes a un Tensor de PyTorch.

In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

training_data = datasets.MNIST( # Crea un objeto MNIST para entrenamiento (subclase de torch.utils.data.Dataset)
    root="data", # ruta donde se almacenan los datos
    train=True, # carga el conjunto de entrenamiento
    download=True,  # descarga el conjunto de datos si no está en el directorio de datos
    transform=ToTensor(), # ToTensor convierte la imagen en un tensor de PyTorch
)

test_data = datasets.MNIST( # Crea un objeto MNIST para Test
    root="data", # ruta donde se guardarán los datos
    train=False, # no carga el conjunto de entrenamiento, sino de Test
    download=True, # descarga el conjunto de datos si es necesario
    transform=ToTensor(), # transforma la imagen en un tensor de PyTorch
)

print (training_data,"\n\n", training_data.data.shape, "\n\n", training_data.data, "\n\n", training_data.targets)


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor() 

 torch.Size([60000, 28, 28]) 

 tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ...,

## 1.2.- Creamos los lotes necesarios para agilizar el procesado del dataset. 

En este caso cargamos el dataset en el DataLoader con un tamaño de lote o Batch de 64.

In [2]:

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

for X, y in test_dataloader:
    print(f"Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: {X.shape}")
    print(f"Shape de y: {y.shape} {y.dtype}")
    break

for X, y in train_dataloader:
    print(f"Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: {X.shape}")
    print(f"Shape de y: {y.shape} {y.dtype}")
    break

Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: torch.Size([64, 1, 28, 28])
Shape de y: torch.Size([64]) torch.int64
Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: torch.Size([64, 1, 28, 28])
Shape de y: torch.Size([64]) torch.int64


## 1.3.- Definicion de la Red Neuronal

Definimos la clase a la que pertenecerá nuestro modelo mediante los métodos "constructor (init)" y "forward"y de la clase nn.Module, detallando los parámetros que vamos a incluir en nuestro modelo de Red Neuronal.

In [3]:

class NeuralNetwork(nn.Module): # Clase que hereda de nn.Module y define la arquitectura de la red
    def __init__(self): # Constructor de la clase
        super().__init__() # Llama al constructor de la clase padre
        self.flatten = nn.Flatten() # Capa de aplanamiento de la imagen (28x28 -> 784)
        self.linear_relu_stack = nn.Sequential( # Secuencia de capas lineales y funciones de activación ReLU
            nn.Linear(28*28, 512), # Capa de entrada con 784 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa de entrada
            nn.Linear(512, 512), # Capa oculta totalmente conectada con 512 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(512, 10) # Capa de salida con 512 entradas y 10 salidas
        )

    def forward(self, x): # Método que define el flujo de datos a través de la red
        x = self.flatten(x) # Aplana la imagen
        logits = self.linear_relu_stack(x) # Pasa los datos a través de la secuencia de capas
        return logits # Devuelve los logits (salida sin activación)
    
model = NeuralNetwork() # Instancia del modelo

## 1.4.- Optimización y entrenamiento

### Definimos el Optimizador y la Función de Pérdida para nuestro modelo. 

In [4]:

loss_fn = nn.CrossEntropyLoss() # Función de pérdida
optimizer = torch.optim.SGD( # Optimizador de descenso de gradiente estocástico
    model.parameters(), # Parámetros del modelo a optimizar
    lr=0.001 # Tasa de aprendizaje
    )


### Definimos el método Train para entrenar nuestro modelo

Para cada lote el método itera sobre los datos del lote o batch, establece una predicción, calcula la pérdida y activa el backpropagation reseteando los gradientes, calculando el nuevo gradiente de la función de pérdida y actualizando los parametros de peso y coste.

In [5]:
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu" 
)
print(f"Using {device} device")

model = model.to(device) # Mueve el modelo a la GPU si está disponible
print(model)


def train(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset) # Número de muestras en el conjunto de datos
    
    model.train() # Pone el modelo en modo de entrenamiento
    for batch_num, (X, y) in enumerate(dataloader): # Itera sobre los lotes de datos, para cada uno:
        X, y = X.to(), y.to(device) # Mueve el array de datos y las etiquetas al dispositivo

        pred = model(X) # Genera predicciones
        loss = loss_fn(pred, y) # Calcula la pérdida para ese lote

        # Backpropagation
        optimizer.zero_grad() # Resetea los gradientes
        loss.backward() # Calcula el gradiente de la función de pérdida
        optimizer.step() # Actualiza los parámetros

        if batch_num % 100 == 0: # Cada 100 lotes imprime el progreso
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Definimos el metodo Test para evaluar nuestro modelo en el conjunto de prueba

In [6]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # Pone el modelo en modo de evaluación
    test_loss, correct = 0, 0
    with torch.no_grad(): # Desactiva el cálculo de gradientes para el siguiente bloque de código
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item() # Acumula la pérdida
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # Acumula el número de aciertos [1]
    test_loss /= num_batches # Calcula la pérdida promedio por lote
    correct /= size # Calcula la exactitud (número de aciertos / número total de muestras)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Entrenando nuestro modelo

Definimos los "Epochs" o iteraciones que queremos que el modelo realice para su entrenamiento. Para cada Epoch se recorre el conjunto de datos dividido el lotes, realizando el ajuste de parámetros según lo definido en el BackPropagation del método Train.

In [7]:
epochs = 10 # Número de epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Fin")

Epoch 1
-------------------------------
loss: 2.305285  [   64/60000]
loss: 2.299359  [ 6464/60000]
loss: 2.294745  [12864/60000]
loss: 2.285345  [19264/60000]
loss: 2.287533  [25664/60000]
loss: 2.289736  [32064/60000]
loss: 2.283995  [38464/60000]
loss: 2.284653  [44864/60000]
loss: 2.272384  [51264/60000]
loss: 2.256912  [57664/60000]
Test Error: 
 Accuracy: 32.1%, Avg loss: 2.264708 

Epoch 2
-------------------------------
loss: 2.265451  [   64/60000]
loss: 2.255053  [ 6464/60000]
loss: 2.257751  [12864/60000]
loss: 2.232626  [19264/60000]
loss: 2.244242  [25664/60000]
loss: 2.244771  [32064/60000]
loss: 2.231841  [38464/60000]
loss: 2.243934  [44864/60000]
loss: 2.220022  [51264/60000]
loss: 2.200363  [57664/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 2.208418 

Epoch 3
-------------------------------
loss: 2.211216  [   64/60000]
loss: 2.191802  [ 6464/60000]
loss: 2.204649  [12864/60000]
loss: 2.153309  [19264/60000]
loss: 2.178839  [25664/60000]
loss: 2.174572  [32064/600

#### Comprobamos que en este modelo, con 1 capa de entrada, una capa oculta totalmente conectada de 512 neuronas, y una capa de salida, dividiendo el dataset en lotes de 64 muestras, y con una tasa de aprendizaje de 0.001, tras realizar 10 iteraciones de entrenamiento, la exactitud conseguida es del 84%

### Modificamos los valores de la red neuronal, modificando la capa oculta lineal por una con 512 entradas y 64 salidas, otra con 64 entradas y 128 salidas y otra más con 128 entradas y 512 salidas, para conectar con la capa de salida de 512 entradas y 10 salidas. Modificamos el Learning Rate a 0.01. 

In [12]:
class NeuralNetwork_2(nn.Module): # Clase que hereda de nn.Module y define la arquitectura de la red
    def __init__(self): # Constructor de la clase
        super().__init__() # Llama al constructor de la clase padre
        self.flatten = nn.Flatten() # Capa de aplanamiento de la imagen (28x28 -> 784)
        self.linear_relu_stack = nn.Sequential( # Secuencia de capas lineales y funciones de activación ReLU
            nn.Linear(28*28, 512), # Capa de entrada con 784 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa de entrada
            nn.Linear(512, 64), # Capa oculta con 512 entradas y 64 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(64, 128), # Capa oculta con 64 entradas y 128 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(128, 512), # Capa oculta con 128 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(512, 10) # Capa de salida con 512 entradas y 10 salidas
        )

    def forward(self, x): # Método que define el flujo de datos a través de la red
        x = self.flatten(x) # Aplana la imagen
        logits = self.linear_relu_stack(x) # Pasa los datos a través de la secuencia de capas
        return logits # Devuelve los logits (salida sin activación)
    
model_2 = NeuralNetwork_2() # Instancia del modelo

optimizer = torch.optim.SGD( # Optimizador de descenso de gradiente estocástico
    model_2.parameters(), # Parámetros del modelo a optimizar
    lr=0.001 # Tasa de aprendizaje
    )

In [13]:
epochs = 10 # Número de epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_2, loss_fn, optimizer)
    test(test_dataloader, model_2, loss_fn)
print("Fin")

Epoch 1
-------------------------------
loss: 2.299796  [   64/60000]
loss: 2.299484  [ 6464/60000]
loss: 2.309227  [12864/60000]
loss: 2.298249  [19264/60000]
loss: 2.300488  [25664/60000]
loss: 2.301455  [32064/60000]
loss: 2.299978  [38464/60000]
loss: 2.312638  [44864/60000]
loss: 2.300023  [51264/60000]
loss: 2.297873  [57664/60000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.300479 

Epoch 2
-------------------------------
loss: 2.297742  [   64/60000]
loss: 2.297847  [ 6464/60000]
loss: 2.306180  [12864/60000]
loss: 2.296951  [19264/60000]
loss: 2.300315  [25664/60000]
loss: 2.300670  [32064/60000]
loss: 2.296892  [38464/60000]
loss: 2.310769  [44864/60000]
loss: 2.298865  [51264/60000]
loss: 2.294949  [57664/60000]
Test Error: 
 Accuracy: 11.3%, Avg loss: 2.298427 

Epoch 3
-------------------------------
loss: 2.295634  [   64/60000]
loss: 2.296136  [ 6464/60000]
loss: 2.303362  [12864/60000]
loss: 2.295328  [19264/60000]
loss: 2.299688  [25664/60000]
loss: 2.299629  [32064/600

#### Comprobamos que a pesar de aumentar drasticamente las capas los resultados no solo no mejoran sino que empeoran considerablemente. Parece que el aumento "random" de capas y neuronas ha provocado un problema de overfitting. 

### Modificamos el dataloader aumentando el tamaño de batch a 100 e incluyendo una mezcla en la carga de datos de entrenamiento que se supone que ayuda a evitar el overfitting, además de definir el número de procesadores. Comprobamos que el resultado no ha variado, el modelo sigue ofreciendo resultados pesimos.

In [19]:
import multiprocessing

class NeuralNetwork_3(nn.Module): # Clase que hereda de nn.Module y define la arquitectura de la red
    def __init__(self): # Constructor de la clase
        super().__init__() # Llama al constructor de la clase padre
        self.flatten = nn.Flatten() # Capa de aplanamiento de la imagen (28x28 -> 784)
        self.linear_relu_stack = nn.Sequential( # Secuencia de capas lineales y funciones de activación ReLU
            nn.Linear(28*28, 512), # Capa de entrada con 784 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa de entrada
            nn.Linear(512, 64), # Capa oculta con 512 entradas y 64 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(64, 128), # Capa oculta con 64 entradas y 128 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(128, 512), # Capa oculta con 128 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(512, 10) # Capa de salida con 512 entradas y 10 salidas
        )

    def forward(self, x): # Método que define el flujo de datos a través de la red
        x = self.flatten(x) # Aplana la imagen
        logits = self.linear_relu_stack(x) # Pasa los datos a través de la secuencia de capas
        return logits # Devuelve los logits (salida sin activación)
    
model_3 = NeuralNetwork_3() # Instancia del modelo

num_workers = multiprocessing.cpu_count()-1
print ("num_workers: ", num_workers)
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=False, num_workers=num_workers)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=num_workers)


optimizer = torch.optim.SGD( # Optimizador de descenso de gradiente estocástico
    model_3.parameters(), # Parámetros del modelo a optimizar
    lr=0.001 # Tasa de aprendizaje
    )

epochs = 10 # Número de epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_3, loss_fn, optimizer)
    test(test_dataloader, model_3, loss_fn)
print("Fin")

num_workers:  7
Epoch 1
-------------------------------
loss: 2.295583  [   32/60000]
loss: 2.301068  [ 3232/60000]
loss: 2.294047  [ 6432/60000]
loss: 2.290294  [ 9632/60000]
loss: 2.299824  [12832/60000]
loss: 2.298966  [16032/60000]
loss: 2.303476  [19232/60000]
loss: 2.299732  [22432/60000]
loss: 2.306796  [25632/60000]
loss: 2.291769  [28832/60000]
loss: 2.307497  [32032/60000]
loss: 2.316036  [35232/60000]
loss: 2.301544  [38432/60000]
loss: 2.296035  [41632/60000]
loss: 2.309145  [44832/60000]
loss: 2.302687  [48032/60000]
loss: 2.302456  [51232/60000]
loss: 2.302650  [54432/60000]
loss: 2.288232  [57632/60000]
Test Error: 
 Accuracy: 11.3%, Avg loss: 2.298427 

Epoch 2
-------------------------------
loss: 2.291005  [   32/60000]
loss: 2.298913  [ 3232/60000]
loss: 2.293610  [ 6432/60000]
loss: 2.285895  [ 9632/60000]
loss: 2.298414  [12832/60000]
loss: 2.299239  [16032/60000]
loss: 2.300195  [19232/60000]
loss: 2.295631  [22432/60000]
loss: 2.302909  [25632/60000]
loss: 2.2833

#### Vemos que la Exactitud no ha mejorado, por lo que optamos por realizar cambios en otros hiperparametros como el tamaño y número de capas.

In [20]:

class NeuralNetwork_4(nn.Module): # Clase que hereda de nn.Module y define la arquitectura de la red
    def __init__(self): # Constructor de la clase
        super().__init__() # Llama al constructor de la clase padre
        self.flatten = nn.Flatten() # Capa de aplanamiento de la imagen (28x28 -> 784)
        self.linear_relu_stack = nn.Sequential( # Secuencia de capas lineales y funciones de activación ReLU
            nn.Linear(28*28, 1024), # Capa de entrada con 784 entradas y 1024 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa de entrada
            nn.Linear(1024, 1024), # Capa oculta totalmente conectada con 1024 entradas y 1024 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(1024, 1024), # Capa oculta totalmente conectada con 1024 entradas y 1024 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(1024, 10) # Capa de salida con 1024 entradas y 10 salidas
        )

    def forward(self, x): # Método que define el flujo de datos a través de la red
        x = self.flatten(x) # Aplana la imagen
        logits = self.linear_relu_stack(x) # Pasa los datos a través de la secuencia de capas
        return logits # Devuelve los logits (salida sin activación)
    
model_4 = NeuralNetwork_4() # Instancia del modelo

# num_workers = multiprocessing.cpu_count()-1
# print (num_workers)
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

loss_fn = nn.CrossEntropyLoss() # Función de pérdida
optimizer = torch.optim.SGD( # Optimizador de descenso de gradiente estocástico
    model_4.parameters(), # Parámetros del modelo a optimizar
    lr=0.01 # Tasa de aprendizaje
    )

epochs = 10 # Número de epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_4, loss_fn, optimizer)
    test(test_dataloader, model_4, loss_fn)
print("Fin")

Epoch 1
-------------------------------
loss: 2.302575  [   64/60000]
loss: 2.287288  [ 6464/60000]
loss: 2.274900  [12864/60000]
loss: 2.235814  [19264/60000]
loss: 2.210763  [25664/60000]
loss: 2.147830  [32064/60000]
loss: 1.985685  [38464/60000]
loss: 1.857762  [44864/60000]
loss: 1.366880  [51264/60000]
loss: 1.032778  [57664/60000]
Test Error: 
 Accuracy: 77.1%, Avg loss: 0.943124 

Epoch 2
-------------------------------
loss: 1.034156  [   64/60000]
loss: 0.753216  [ 6464/60000]
loss: 0.720700  [12864/60000]
loss: 0.607432  [19264/60000]
loss: 0.575822  [25664/60000]
loss: 0.470430  [32064/60000]
loss: 0.398287  [38464/60000]
loss: 0.531275  [44864/60000]
loss: 0.527048  [51264/60000]
loss: 0.514476  [57664/60000]
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.424852 

Epoch 3
-------------------------------
loss: 0.486511  [   64/60000]
loss: 0.327497  [ 6464/60000]
loss: 0.355538  [12864/60000]
loss: 0.406940  [19264/60000]
loss: 0.361325  [25664/60000]
loss: 0.368834  [32064/600

#### Comprobamos que la Exactitud ha mejorado, por lo que buscamos alguna técnica adicional para ir mejorando los resultados. 

### Probamos alguna técnica extraída del informe "Techniques to Improve Our MNIST Accuracy Without Using CNNs" publicado en el blog https://medium.com/@anderaquerretamontoro/99-46-accuracy-on-mnist-without-cnn-712042530420 como  por ejemplo utilizar un learnig rate variable aumentando los Epochs.

In [22]:

class NeuralNetwork_5(nn.Module): # Clase que hereda de nn.Module y define la arquitectura de la red
    def __init__(self): # Constructor de la clase
        super().__init__() # Llama al constructor de la clase padre
        self.flatten = nn.Flatten() # Capa de aplanamiento de la imagen (28x28 -> 784)
        self.linear_relu_stack = nn.Sequential( # Secuencia de capas lineales y funciones de activación ReLU
            nn.Linear(28*28, 1024), # Capa de entrada con 784 entradas y 1024 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa de entrada
            nn.Linear(1024, 1024), # Capa oculta totalmente conectada con 1024 entradas y 1024 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(1024, 1024), # Capa oculta totalmente conectada con 1024 entradas y 1024 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(1024, 10) # Capa de salida con 1024 entradas y 10 salidas
        )

    def forward(self, x): # Método que define el flujo de datos a través de la red
        x = self.flatten(x) # Aplana la imagen
        logits = self.linear_relu_stack(x) # Pasa los datos a través de la secuencia de capas
        return logits # Devuelve los logits (salida sin activación)
    
model_5 = NeuralNetwork_5() # Instancia del modelo
optimizer = torch.optim.SGD(model_5.parameters(), lr=0.1, weight_decay=1e-6, momentum=0.9)

# Learning Rate Annealing (LRA) scheduling
# lr = 0.1     if epoch < 25
# lr = 0.01    if 25 <= epoch < 50
# lr = 0.001   if epoch >= 50
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50], gamma=0.1)


# Start training
epochs = 75 # Número de epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_5, loss_fn, optimizer)
    test(test_dataloader, model_5, loss_fn)
    scheduler.step()
    print("\tLearningRate: ", optimizer.param_groups[0]['lr'])
print("Fin")




Epoch 1
-------------------------------
loss: 2.304788  [   64/60000]
loss: 0.313561  [ 6464/60000]
loss: 0.306146  [12864/60000]
loss: 0.342958  [19264/60000]
loss: 0.109508  [25664/60000]
loss: 0.301690  [32064/60000]
loss: 0.082986  [38464/60000]
loss: 0.162539  [44864/60000]
loss: 0.242330  [51264/60000]
loss: 0.242473  [57664/60000]
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.173275 

	LR:  0.1
Epoch 2
-------------------------------
loss: 0.183583  [   64/60000]
loss: 0.147915  [ 6464/60000]
loss: 0.081132  [12864/60000]
loss: 0.182753  [19264/60000]
loss: 0.335545  [25664/60000]
loss: 0.169885  [32064/60000]
loss: 0.051537  [38464/60000]
loss: 0.189772  [44864/60000]
loss: 0.181064  [51264/60000]
loss: 0.121322  [57664/60000]
Test Error: 
 Accuracy: 96.2%, Avg loss: 0.134185 

	LR:  0.1
Epoch 3
-------------------------------
loss: 0.115540  [   64/60000]
loss: 0.130552  [ 6464/60000]
loss: 0.102846  [12864/60000]
loss: 0.202242  [19264/60000]
loss: 0.032586  [25664/60000]
loss: 

#### Comprobamos que la Precisión ha subido hasta el 98.6%. Aunque posiblemente el hecho de aumentar los Epochs y modificar el learning rate también hubiese mejorado los resultados en el modelo original, el hecho de ir optimizando el número de capas, y que el learning rate sea dinámico durante el entrenamiento posiblemente ha hecho que el modelo consiga una fiabilidad alta. 

# <span style="color:cornflowerblue"><u>2. Crea otro modelo convolucional como el del ejemplo propuesto intentando mejorar la exactitud.</u> </span>

## Crearemos un modelo convolucional sobre el dataset FashionMnist intentando optimizar los hiperparámetros para mejorar sus valores de exactitud.

## 2.1 Importamos las librerías necesarias

Se importan las librerias de PyTorch necesarias para construir la Red Neuronal Convolucional. 

Se descarga el dataset FashionMnist separando el grupo de entrenamiento y test y transformando las imagenes a un Tensor de PyTorch.

Cargamos el dataset en el DataLoader con un tamaño de lote o Batch de 64.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.nn import functional as F

# Definimos las transformaciones para preprocesado de las imágenes
transform = transforms.Compose([transforms.ToTensor(),])

# Cargamos el dataset Fashion MNIST
train_Fdata = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
test_Fdata = datasets.FashionMNIST('./data', train=False, download=True, transform=transform)

# Creamos los dataloaders
train_Floader = DataLoader(train_Fdata, batch_size=64, shuffle=True)
test_Floader = DataLoader(test_Fdata, batch_size=64, shuffle=False)

print (train_Fdata,"\n\n", train_Fdata.data.shape, "\n\n", "\n\n", train_Fdata.targets)

for X, y in test_Floader:
    print(f"Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: {X.shape}")
    print(f"Shape de y: {y.shape} {y.dtype}")
    break

for X, y in train_Floader:
    print(f"Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: {X.shape}")
    print(f"Shape de y: {y.shape} {y.dtype}")
    break

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           ) 

 torch.Size([60000, 28, 28]) 

 

 tensor([9, 0, 0,  ..., 3, 0, 5])
Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: torch.Size([64, 1, 28, 28])
Shape de y: torch.Size([64]) torch.int64
Shape de X [N(numero de muestras), C(canales de color), H(altura), W(anchura)]: torch.Size([64, 1, 28, 28])
Shape de y: torch.Size([64]) torch.int64


## 2.2 Definimos la red 

Definimos la clase a la que pertenecerá nuestro modelo mediante los métodos "constructor (init)" y "forward"y de la clase nn.Module, detallando los parámetros que vamos a incluir en nuestro modelo de Red Neuronal Convolucional.

In [3]:


class CNN(nn.Module): # Definimos la red neuronal convolucional
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # primera capa convolucional, 1 canal de color, 32 features, ventana de 3x3. Tamaño 32*28*28
    self.pool = nn.MaxPool2d(2, 2) # capa de maxpooling, reduce el tamaño de la imagen a la mitad manteniendo las features
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # segunda capa convolucional, 32 features de entrada, 64 de salida, ventana de 3x3. Tamaño de la imagen 64*14*14
    self.fc1 = nn.Linear(7 * 7 * 64, 128) # capa linear de entrada, toma un tensor aplanado de 7*7*64 en la entrada y devuelve 128 en la salida
    self.fc2 = nn.Linear(128, 10) # capa linear de salida, toma un tensor de 128 en la entrada y devuelve 10 en la salida que son el número de etiquetas del dataset

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x))) # paso por 1ª capa convolucional del batch, aplica función de activación Relu y reduce tamaño con maxpooling (1,28,28->32,14,14)
    x = self.pool(F.relu(self.conv2(x))) # paso por 2ª capa convolucional la salida de la 1ª capa, aplica función de activación Relu y reduce tamaño con maxpooling (32,14,14->64,7,7)
    x = x.view(-1, 7 * 7 * 64) # Aplanamiento tras las capas convolucionales (flatten)
    x = F.relu(self.fc1(x)) # paso por 1ª capa linear de la red neuronal, 3136 neuronas de entrada y 128 de salida, aplica funcion de activación Relu (3136,128)
    x = F.log_softmax(self.fc2(x), dim=1)  # paso por 2ª capa de la red neuronal, 128 neuronas de entrada y 10 de salida, correspondientes con las etiquetas del dataset.
    return x
  
modelConv = CNN() # Instanciamos la red neuronal

## 2.3.- Optimización y entrenamiento

### Definimos el Optimizador y la Función de Pérdida para nuestro modelo. 

Definimos la función de perdida (CrossEntropyLoss) y el optimizador, en este caso Adam, que es una variante del descenso de gradiente estocástico que calcula tasas de aprendizaje individuales para diferentes parámetros.

In [4]:
lossFn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(modelConv.parameters())

### Definimos el método Train para entrenar nuestro modelo

Para cada lote el método itera sobre los datos del lote o batch, establece una predicción, calcula la pérdida y activa el backpropagation reseteando los gradientes, calculando el nuevo gradiente de la función de pérdida y actualizando los parametros de peso y coste.

In [5]:
modelConv.train() # Ponemos el modelo en modo entrenamiento 

for epoch in range(5): # Definimos 10 epochs
  
  for i, (images, labels) in enumerate(train_Floader):
    # Forward pass
    outputs = modelConv(images) # Genera predicciones
    loss = lossFn(outputs, labels) # Calcula la perdida en la iteración

    # Backpropagation
    optim.zero_grad() # Resetea los gradientes
    loss.backward() # Calcula el gradiente de la función de pérdida
    optim.step() # Actualiza los parámetros de pesos y sesgos

    if (i + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{5}], Step [{i+1}/{len(train_Floader)}], Loss: {loss.item():.4f}')
      
      

Epoch [1/10], Step [100/938], Loss: 0.3038
Epoch [1/10], Step [200/938], Loss: 0.3650
Epoch [1/10], Step [300/938], Loss: 0.5338
Epoch [1/10], Step [400/938], Loss: 0.2693
Epoch [1/10], Step [500/938], Loss: 0.3609
Epoch [1/10], Step [600/938], Loss: 0.4540
Epoch [1/10], Step [700/938], Loss: 0.3548
Epoch [1/10], Step [800/938], Loss: 0.4457
Epoch [1/10], Step [900/938], Loss: 0.3956
Epoch [2/10], Step [100/938], Loss: 0.2378
Epoch [2/10], Step [200/938], Loss: 0.3073
Epoch [2/10], Step [300/938], Loss: 0.3441
Epoch [2/10], Step [400/938], Loss: 0.1632
Epoch [2/10], Step [500/938], Loss: 0.2529
Epoch [2/10], Step [600/938], Loss: 0.3449
Epoch [2/10], Step [700/938], Loss: 0.2809
Epoch [2/10], Step [800/938], Loss: 0.2512
Epoch [2/10], Step [900/938], Loss: 0.2544
Epoch [3/10], Step [100/938], Loss: 0.3556
Epoch [3/10], Step [200/938], Loss: 0.2067
Epoch [3/10], Step [300/938], Loss: 0.3696
Epoch [3/10], Step [400/938], Loss: 0.3144
Epoch [3/10], Step [500/938], Loss: 0.2064
Epoch [3/10

## 2.4 Evaluación del modelo



In [6]:
with torch.no_grad(): # Deshabilita el calculo de gradientes
  correct = 0
  total = 0
  for images, labels in test_Floader:
    outputs = modelConv(images)
    _, pred = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 91.27%


#### Comprobamos que la exactitud en este dataset es más baja, intentamos modificar los hiperparámetros para lograr un mejor valor

In [10]:

class CNN_2(nn.Module): # Definimos la red neuronal convolucional
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # primera capa convolucional, 1 canal de color, 32 features, ventana de 3x3. Tamaño 32*28*28
    self.pool = nn.MaxPool2d(2, 2) # capa de maxpooling, reduce el tamaño de la imagen a la mitad manteniendo las features
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # segunda capa convolucional, 32 features de entrada, 64 de salida, ventana de 3x3. Tamaño de la imagen 64*14*14
    self.fc1 = nn.Linear(7 * 7 * 64, 128) # capa linear de entrada, toma un tensor aplanado de 7*7*64 en la entrada y devuelve 128 en la salida
    self.fc2 = nn.Linear(128, 10) # capa linear de salida, toma un tensor de 128 en la entrada y devuelve 10 en la salida que son el número de etiquetas del dataset

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x))) # paso por 1ª capa convolucional del batch, aplica función de activación Relu y reduce tamaño con maxpooling (1,28,28->32,14,14)
    x = self.pool(F.relu(self.conv2(x))) # paso por 2ª capa convolucional la salida de la 1ª capa, aplica función de activación Relu y reduce tamaño con maxpooling (32,14,14->64,7,7)
    x = x.view(-1, 7 * 7 * 64) # Aplanamiento tras las capas convolucionales (flatten)
    x = F.sigmoid(self.fc1(x)) # paso por 1ª capa linear de la red neuronal, 3136 neuronas de entrada y 128 de salida, aplica funcion de activación Relu (3136,128)
    x = F.log_softmax(self.fc2(x), dim=1)  # paso por 2ª capa de la red neuronal, 128 neuronas de entrada y 10 de salida, correspondientes con las etiquetas del dataset.
    return x
  
modelConv_2 = CNN_2() # Instanciamos la red neuronal

lossFn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(modelConv_2.parameters(), lr=0.1, weight_decay=1e-6, momentum=0.9)

# Learning Rate Annealing (LRA) scheduling
# lr = 0.1     if epoch < 25
# lr = 0.01    if 25 <= epoch < 50
# lr = 0.001   if epoch >= 50
sched = torch.optim.lr_scheduler.MultiStepLR(optim, milestones=[4, 8], gamma=0.1)



modelConv_2.train() # Ponemos el modelo en modo entrenamiento 

for epoch in range(12): # Definimos 10 epochs
  
  for i, (images, labels) in enumerate(train_Floader):
    # Forward pass
    outputs = modelConv_2(images) # Genera predicciones
    loss = lossFn(outputs, labels) # Calcula la perdida en la iteración

    # Backpropagation
    optim.zero_grad() # Resetea los gradientes
    loss.backward() # Calcula el gradiente de la función de pérdida
    optim.step() # Actualiza los parámetros de pesos y sesgos
    
    

    if (i + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{12}], Step [{i+1}/{len(train_Floader)}], Loss: {loss.item():.4f}')
  sched.step()
  print("\tLearningRate: ", optim.param_groups[0]['lr'])    
      

Epoch [1/12], Step [100/938], Loss: 0.9874
Epoch [1/12], Step [200/938], Loss: 0.6368
Epoch [1/12], Step [300/938], Loss: 0.6046
Epoch [1/12], Step [400/938], Loss: 0.6895
Epoch [1/12], Step [500/938], Loss: 0.3520
Epoch [1/12], Step [600/938], Loss: 0.4296
Epoch [1/12], Step [700/938], Loss: 0.3135
Epoch [1/12], Step [800/938], Loss: 0.3433
Epoch [1/12], Step [900/938], Loss: 0.2742
	LearningRate:  0.1
Epoch [2/12], Step [100/938], Loss: 0.3056
Epoch [2/12], Step [200/938], Loss: 0.4752
Epoch [2/12], Step [300/938], Loss: 0.3638
Epoch [2/12], Step [400/938], Loss: 0.2030
Epoch [2/12], Step [500/938], Loss: 0.2879
Epoch [2/12], Step [600/938], Loss: 0.1926
Epoch [2/12], Step [700/938], Loss: 0.2124
Epoch [2/12], Step [800/938], Loss: 0.2833
Epoch [2/12], Step [900/938], Loss: 0.2016
	LearningRate:  0.1
Epoch [3/12], Step [100/938], Loss: 0.2480
Epoch [3/12], Step [200/938], Loss: 0.2326
Epoch [3/12], Step [300/938], Loss: 0.2321
Epoch [3/12], Step [400/938], Loss: 0.2527
Epoch [3/12], 

#### Evaluación del modelo modificado

In [11]:
with torch.no_grad(): # Deshabilita el calculo de gradientes
  correct = 0
  total = 0
  for images, labels in test_Floader:
    outputs = modelConv_2(images)
    _, pred = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 92.13%


#### Comprobamos que cambiando el optimizador a uno con tasa de aprendizaje variable y añadiendo un par de Epochs y modificando la función de activación a Sigmoide, la Exactitud mejora casi un  punto hasta el 92,13%